# System Public API Examples

This notebook provides basic examples of how to use System's public API. Full documentation is available on our [API documentation site](https://api-docs.system.com/).

Running these examples requires you to have an API key, provided by System. Please contact us at [api@system.com](mailto:api@system.com) if you are interested in exploring our public API.

## 1. Import libraries and define helper functions

In [25]:
!pip install api-client==1.30.0 --extra-index-url https://pypi.fury.io/systeminc/

Looking in indexes: https://pypi.org/simple, https://pypi.fury.io/systeminc/

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [26]:
import api_client
import os

In [27]:
def pprint_topic(t):
    print(f"{t.name} (ID: {t.id}, Wikidata ID: {t.wikidata_id})")
    
def pprint_determinant(d):
    print(f"{d.topic.name} (ID: {d.id}, number of findings: {d.num_findings}, median effect size: {d.median_effect_size})")
    
def pprint_relationship(r):
    print(f"{r.source_topic.name} -> {r.target_topic.name} (ID: {r.id}, number of findings: {r.num_findings}, median effect size: {r.median_effect_size})")
    
def pprint_study(s):
    print(f"'{s.title}', {s.publish_date} (ID: {s.id}, DOI: {s.doi})")
    print(f"{s.summary}")
    
def pprint_variable(v):
		print(f"{v.name} (ID: {v.id}, reported in {v.num_studies} studies)")
  
def pprint_finding(f):
	print(f"{f.summary}")

## 2. Create a service client and initialize endpoints

In [48]:
%env YOUR_API_KEY=<YOUR_API_KEY>

env: YOUR_API_KEY=<YOUR_API_KEY>


In [29]:
API_KEY = os.environ.get("YOUR_API_KEY")
if not API_KEY:
	raise ValueError("API key is missing")

system_client = api_client.ApiClient(header_name="x-api-key", header_value=API_KEY)

topics_api = api_client.api.topics_api.TopicsApi(system_client)
relationships_api = api_client.api.relationships_api.RelationshipsApi(system_client)
studies_api = api_client.api.studies_api.StudiesApi(system_client)
variables_api = api_client.api.variables_api.VariablesApi(system_client)
findings_api = api_client.api.statistical_findings_api.StatisticalFindingsApi(system_client)

## 3. Search for and retrieve a topic

In [30]:
def search_topic_by_name(topic_name: str):
  response = topics_api.get_topics(search=f"name:{topic_name}")
  return response.data
  

In [31]:
topics = search_topic_by_name(topic_name="osteoporosis")
for t in topics:
	pprint_topic(t)

postmenopausal osteoporosis (ID: ZwLPJsQaKE, Wikidata ID: Q55950294)
idiopathic juvenile osteoporosis (ID: TzD7fS4xV9, Wikidata ID: Q6318965)
osteoporosis (ID: YCCUBPRxoV, Wikidata ID: Q165328)
osteoporosis-pseudoglioma syndrome (ID: kELO0u-WL3, Wikidata ID: Q32136402)


In [32]:
def search_topic_by_wikidata_id(wikidata_id: str):
  response = topics_api.get_topics(filter=f"wikidata_id:{wikidata_id}")
  return response.data

In [33]:
topics_2 = search_topic_by_wikidata_id(wikidata_id="Q55950294")
for t in topics_2:
	pprint_topic(t)

postmenopausal osteoporosis (ID: ZwLPJsQaKE, Wikidata ID: Q55950294)


In [34]:
def get_topic_by_id(topic_id: str):
  return topics_api.get_topic_by_id(topic_id)

In [35]:
topic_id = "YCCUBPRxoV" # Topic ID for 'osteoporosis'
topic = get_topic_by_id(topic_id=topic_id) 
pprint_topic(topic)

osteoporosis (ID: YCCUBPRxoV, Wikidata ID: Q165328)


## 4. Retrieve variables and determinants associated with a topic

In [36]:
def get_variables_by_topic_id(topic_id: str):
  response = variables_api.get_variables(filter=f"topic.id:{topic_id}", limit=15)
  return response.data

In [37]:
variable_id = get_variables_by_topic_id(topic_id=topic_id)
for v in variable_id:
	pprint_variable(v)

osteoporosis (ID: WE3gMwW_XO, reported in 580 studies)
risk of osteoporosis (ID: QPOd9wGo9B, reported in 118 studies)
Osteoporosis (ID: AnzcmtGO8d, reported in 110 studies)
prevalence of osteoporosis (ID: mt2BZa1_7e, reported in 57 studies)
osteoporosis risk (ID: CXrwLUzJjH, reported in 40 studies)
incidence of osteoporosis (ID: j4OcL-v4SY, reported in 23 studies)
Prevalence of osteoporosis (ID: skWIhK5ILX, reported in 21 studies)
presence of osteoporosis (ID: 7u6BoN0l2m, reported in 15 studies)
osteoporosis prevalence (ID: b3vcuk9SKT, reported in 14 studies)
risk of developing osteoporosis (ID: Fq5lEpGOeV, reported in 13 studies)
Patients with osteoporosis [vs Patients without osteoporosis] (ID: m-0X7jCJx1, reported in 12 studies)
osteoporosis diagnosis (ID: 8btd5AURGn, reported in 12 studies)
incident osteoporosis (ID: evB2qy9Z3j, reported in 10 studies)
occurrence of osteoporosis (ID: 1h1dl0m0bL, reported in 10 studies)
osteoporosis treatment (ID: D1selZO3Be, reported in 10 studies)

In [38]:
def get_determinants_by_topic_id(topic_id: str):
  response = topics_api.get_topic_determinants(topic_id, filter="num_significant_findings:>1", sort="median_effect_size:desc")
  return response.data

In [39]:
determinants = get_determinants_by_topic_id(topic_id=topic_id)
for d in determinants:
	pprint_determinant(d)

postmenopause (ID: ELjr0Yjdsc|YCCUBPRxoV, number of findings: 10, median effect size: 1.0974809933366865)
sex (ID: wWbfsY5E0d|YCCUBPRxoV, number of findings: 72, median effect size: 0.7765715424921846)
age group (ID: HejFSI-X_z|YCCUBPRxoV, number of findings: 31, median effect size: 0.5117541228885152)
ethnicity and race (ID: EhpdS79G1u|YCCUBPRxoV, number of findings: 20, median effect size: 0.47573616584177714)
menarche (ID: W_5xNoEZX9|YCCUBPRxoV, number of findings: 3, median effect size: 0.44730108995921114)
fibroblast growth factor family (ID: i4pV-bfaG8|YCCUBPRxoV, number of findings: 7, median effect size: 0.33737861926793683)
TNF receptor superfamily member 11b (ID: Lok8p6Sgq2|YCCUBPRxoV, number of findings: 8, median effect size: 0.1513049630037923)


## 5. Retrieve a relationship

In [40]:
def get_relationship_by_id(relationship_id: str):
	return relationships_api.get_relationship_by_id(relationship_id)

In [41]:
relationship_id = "ELjr0Yjdsc|YCCUBPRxoV" # Relationship between 'postmenopause' and 'osteoporosis'
relationship = get_relationship_by_id(relationship_id=relationship_id)
pprint_relationship(relationship)

postmenopause -> osteoporosis (ID: ELjr0Yjdsc|YCCUBPRxoV, number of findings: 10, median effect size: 1.0974809933366865)


## 6. Search for and retrieve a study

In [42]:
def get_study_by_doi(doi: str):
	response = studies_api.get_studies(filter=f"doi:{doi}")
	return response.data

In [43]:
study = get_study_by_doi(doi="10.1016/j.bone.2010.03.017")
for s in study:
	pprint_study(s)

'Prevalence and risk factors of osteoporosis in Korea: a community-based cohort study with lumbar spine and hip bone mineral density.', 2010-08-01 (ID: BwNHkghmPt, DOI: 10.1016/j.bone.2010.03.017)
The study population consisted of 1,547 men and 1991 women aged 40 years and older with BMD measurements from a prospective community-based cohort in Korea. Crude prevalence of osteoporosis in the whole subjects (40-79 years old) was 13.1% for men and 24.3% for women by WHO criteria, at any site among lumbar spine, femoral neck or total hip. The mean BMD of studied female subjects after age of 50 was not significantly different from that of Chinese but significantly lower than that of Japanese, non-Hispanic whites, non.Hispanic blacks and Mexican Americans.


## 7. Retrieve statistical findings from relationships or studies

In [44]:
def get_findings_by_relationship_id(relationship_id: str):
  response = relationships_api.get_relationship_statistical_findings(relationship_id)
  return response.data

In [45]:
relationship_findings = get_findings_by_relationship_id("ELjr0Yjdsc|YCCUBPRxoV")
for f in relationship_findings:
	pprint_finding(f)

Postmenopausal RA patients [vs Postmenopausal healthy controls] is associated with a 9.6 unit increase in the mean of Osteoporosis incidence.
post-menopause status is associated with a 632.0% increase in the odds of osteoporosis.
postmenopausal status is associated with a 1059.0% increase in the relative risk of spinal osteoporosis.
Postmenopausal women [vs Premenopausal women] is associated with a -1.8 unit decrease in the mean of Time to develop osteoporosis.
postmenopausal women (age >50) is associated with a 3360.0% increase in the odds of osteopenia/osteoporosis/osteopenia or LBD.
Postmenopausal women [vs Premenopausal women] is associated with a 31.0 unit increase in the mean of Incidence of osteoporosis.
Post-menopausal period [vs Pre-menopausal period] is associated with a 13.0 unit increase in the mean of Prevalence of osteopenia/osteoporosis.
postmenopausal females [vs adult males] is associated with a 27.2 unit increase in the mean of prevalence of osteoporosis.
Postmenopaus

In [46]:
def get_findings_by_study_id(study_id: str):
	response = studies_api.get_study_statistical_findings(study_id)
	return response.data

In [47]:
study_findings = get_findings_by_study_id(study_id="BwNHkghmPt")
for f in study_findings:
	pprint_finding(f)

having more than three offspring (reproductive history) is associated with a 42.0% increase in the odds of osteoporosis.
menarche after age of 16 is associated with a 46.0% increase in the odds of osteoporosis.
post-menopause status is associated with a 632.0% increase in the odds of osteoporosis.
last delivery after age of 30 is associated with a 58.0% increase in the odds of osteoporosis.
more than 17 years since menopause is associated with a 53.0% increase in the odds of osteoporosis.
smoking> or =1 pack/day is associated with a 63.0% increase in the odds of osteoporosis.
past fracture history is associated with a 45.0% increase in the odds of osteoporosis.
calcium intake> or =627.5 mg/day is associated with a -35.0% decrease in the odds of osteoporosis.
monthly income above 500,000 won per household is associated with a -36.0% decrease in the odds of osteoporosis.
college graduate is associated with a -71.0% decrease in the odds of osteoporosis.
